# Final assignment programming 1

###  People with a low income have an increased chance of health risks

## Help during programming
Gitbook
https://fennaf.gitbook.io/bfvm22prog1/ <br>

https://www.ibm.com/docs/en/watson-studio-local/1.2.3?topic=notebooks-markdown-jupyter-cheatsheet

<br>
Pandas and bokeh programming <br>
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.interpolate.html <br>
https://rubberduckdebugging.com/ <br>
https://docs.bokeh.org/en/latest/docs/gallery.html<br>
https://opensourceuom.gitlab.io/blog/post_files/2022-02-23/git-essentials-cheatsheet.pdf

Tutorials about combining data: https://github.com/fenna/BFVM22PROG1/blob/main/tutorials/tutorial_combine_data.ipynb

https://machinelearningmastery.com/statistical-hypothesis-tests-in-python-cheat-sheet/ 

If your data is not normally distributed you might want to look for an alternative. See also https://www.statisticshowto.com/probability-and-statistics/statistics-definitions/parametric-and-non-parametric-data/

We can use pandas.DataFrame.melt to prepare the data for statistical analysis. 
https://fennaf.gitbook.io/bfvm19prog1/data-wrangling/reshape-with-melt

I updated the gitbook (with data exploration part mainly). Please refresh your browser to get the latest version https://fennaf.gitbook.io/bfvm22prog1/


https://docs.bokeh.org/en/latest/docs/examples/basic/data/transform_markers.html <br>
https://docs.bokeh.org/en/latest/docs/user_guide/interaction/linking.html#ug-interaction-linked-panning <br>

In [ ]:
# Pearson correlation?
# libraries, functions, rest
# consider puttting functions into a module

***

## About the data

The data comes from the Lifelines cohort. Lifelines is a large, multi-generational, prospective cohort study that includes over 167,000 participants (10%) from the northern population of the Netherlands. 
Within this cohort study the participants are followed over a 30-year period. Every five years, participants visit one of the Lifelines sites in the northern parts of the Netherlands for an assessment. During these visits, several physical measurements are taken and different biomaterials are collected. As part of the assessment, participants are asked to fill out comprehensive questionnaires. In between assessments, participants are invited to complete follow-up questionnaires approximately once every 1.5 years.

https://coronabarometer.nl/ <br>
https://wiki-lifelines.web.rug.nl/doku.php?id=cohort <br>
https://wiki-lifelines.web.rug.nl/doku.php?id=1a <br>
https://wiki-lifelines.web.rug.nl/doku.php?id=default_variables <br>
https://inzicht.lifelines.nl/index.php <br>

The data is aggregated on zip codes and on age groups in order to preserve the
privacy of participants.

NSES_YEAR = Year for which the NSES score was calculated as mean <BR>
NSES = Neighborhood socio-economic status score according to CBS Statistics
Netherlands, based on inhabitants’ educational level, income and job
prospective as mean
<br>


***

## Libraries:

In [ ]:
import yaml

import pandas as pd
import numpy as np
import seaborn as sns
import hvplot.pandas
import statsmodels.api as sm
import matplotlib.pyplot as plt
from scipy.stats import shapiro

#import bokeh and direct the output to the notebook
from bokeh.io import output_notebook
from bokeh.layouts import gridplot
from bokeh.plotting import figure, output_file, show
output_notebook()

In [ ]:
from Summary_functions_and_code_clean_v11 import DS_Q_Q_Hist, DS_Q_Q_Plot

***

## Load the data

In [ ]:
with open("config.yaml", 'r') as stream:
    config = yaml.safe_load(stream)
zipcodes_general = config['lifelines_zipcode_gen']
zipcodes = config['lifelines_zipcode']
age = config['lifelines_age']

In [ ]:
# Name the dataframes
# Encoding utf_16 is necessary to read the table
zipcode_gen = pd.read_table(zipcodes_general, encoding='utf_16')
zipcodes = pd.read_table(zipcodes, encoding='utf_16')
age = pd.read_table(age, encoding='utf_16')

***

## Cleaning and inspecting the data

In [ ]:
# Inspect dataframe data aggregated on zipcode general
zipcode_gen.head()

In [ ]:
# Inspect zipcode dataframe
zipcodes.head()

In [ ]:
# Inspect dataframe data aggregated on age
age.head()

In [ ]:
age.tail()

In [ ]:
# Check info of dataframe, because there seems to be missing data
age.info()

In [ ]:
f"The number of entries is {len(age)}"

- There are 72 variables (columns) in total in the dataframe <br>
- Except for the first three columns all the datatypes are objects. The values are averages of the age group, so the dtype should be changed to floats.
- Also, the missing values don't register as missing values. Spaces are used to fill in the missing values.

In [ ]:
# Change the datatype from objects to floats
#age['AGE_T1'] = age.AGE_T1.str.replace(',', '.').astype(float)

In [ ]:
# Replace comma's with dots for the averages
age = age.replace(',', '.',regex=True)

In [ ]:
# Replace the whitespaces with NaN values
age = age.replace(r'\s+', np.nan, regex=True)

In [ ]:
#df.replace('G', 1, inplace=True)

In [ ]:
age.tail()

In [ ]:
# Check the total number of missing values
f"The total number of missing values are {age.isnull().sum().sum()}"

In [ ]:
#age.info()

##### Categories with missing numbers are:
Personality <br>
- C_SUM_T1
- A_SUM_T1
- SC_SUM_T1
- I_SUM_T1
- E_SUM_T1
- SD_SUM_T1
- V_SUM_T1 
- D_SUM_T1

Only completed cases, as mean, of the personality questionaire are in the data set

Health and lifestyle:
- ALCOHOL_INTAKE_T1
- KCAL_INTAKE_T1
- PREGNANCIES,
- SLEEP_QUALITY

Only women can get pregnant, so it make sense that there is data for only the females.

In [ ]:
# Uniques and counts for the Age column
age.AGE.value_counts()

Age categories range from 25 to 85 years old and are sperated in male and female<br>

In [ ]:
# Change the dtype of the data frame to floats
age = age.astype(float)

In [ ]:
# First three columns should stay integers
age = age.astype({'AGE':'int', 'GENDER':'int','GROUP_SIZE_CAT':'int'})

In [ ]:
# Check if the changes worked
age.head()

In [ ]:
# Descriptive stats of data frame
age.describe().T

#### Select the columns that might be relevant for the research question

In [ ]:
age_new = age[['AGE','GENDER','EDUCATION_LOWER_T1', 'OSTEOARTHRITIS_T1','NSES','GROUP_SIZE_CAT']]

In [ ]:
# Make a copy of the newly created dataframe
df = age_new.copy()

According to the toolkit of lifelines: <br>
Gender is:
- Male = 1
- Female = 2

In [ ]:
# Change the values of the gender column to their corresponding name
df['GENDER'] = df['GENDER'].replace([1, 2], ['Male', 'Female'])

## Check distribution/normality

### Q_Q plot

A QQ-plot is used to visually determine how close a sample is to a the Normal distribution. If the points fall roughly on the diagonal line, then the samples can be considered to be distributed normal.

In [ ]:
# Another Q_Q plot (DS1 bayesian statistics)
print('Education')
DS_Q_Q_Plot(df.EDUCATION_LOWER_T1)
print('Osteoarthritis')
DS_Q_Q_Plot(df.OSTEOARTHRITIS_T1)


### Histogram

Before assessing the distribution statistically, plot the distribution first.

In [ ]:
# Histogram of education by gender
histogram = df.hvplot.hist('OSTEOARTHRITIS_T1',subplots=True, alpha = 0.4).opts(toolbar=None)
histogram

In [ ]:
histogram = df.hvplot.hist('EDUCATION_LOWER_T1',subplots=True, alpha = 0.4).opts(toolbar=None)
histogram

In [ ]:
print('Education')
DS_Q_Q_Hist(df.EDUCATION_LOWER_T1)
print('Osteoarthritis')
DS_Q_Q_Hist(df.OSTEOARTHRITIS_T1)

Not normally distributed

#### Correlations

See if there correlations amongs the variables: <br>
<b> Heatmap

In [ ]:
df.corr(method='spearman').abs()

In [ ]:
#correlation matrix
# Heatmap
corr = df.corr().abs()
#selection.corr().abs()
sns.heatmap(corr, annot=True)

In [ ]:
from scipy.stats import spearmanr

rho, p = spearmanr(df['EDUCATION_LOWER_T1'],df['OSTEOARTHRITIS_T1'])
print(rho)
print(p)

There is a significant correlation

## Visualization

## Conclusion

***

## Gender

In [ ]:
# Seperate the dataframe into dataframe for male and female
male = df.loc[df['GENDER'] == 'Male']
female = df.loc[df['GENDER'] == 'Female']

In [ ]:
male.corr().abs()

In [ ]:
#correlation matrix
# Heatmap
corr_male = male.corr().abs()
#selection.corr().abs()
sns.heatmap(corr_male, annot=True)

In [ ]:
female.corr().abs()

In [ ]:
#correlation matrix
# Heatmap
corr_female = female.corr().abs()
#selection.corr().abs()
sns.heatmap(corr_female, annot=True)

***

#### Male

In [ ]:
# Q_Q plot of education level of males (statsmodels)
fig = sm.qqplot(male.EDUCATION_LOWER_T1, fit = True, line = '45')
plt.show()

In [ ]:
# Another Q_Q plot (DS1 bayesian statistics)
DS_Q_Q_Plot(male.EDUCATION_LOWER_T1)

With the Bayesian DS1 Q_Q plot you can see that the data falls within a 95% confidence interval. <br>
The data seems to be following the normal line. Indicating that the data might be normally distributed.

#### Female

In [ ]:
# Q_Q_plot of education level of females (statsmodels)
fig = sm.qqplot(female.EDUCATION_LOWER_T1, fit = True, line = '45')
plt.show()

In [ ]:
# Another Q_Q plot (DS1 bayesian statistics)
DS_Q_Q_Plot(female.EDUCATION_LOWER_T1)

Most of the data seems to be in the 95% confidence interval. However it does not seem to be linear, so might not be normally distributed for women.

In [ ]:
# Histogram of education by gender
histogram = df.hvplot.hist('EDUCATION_LOWER_T1', by='GENDER', subplots=True, alpha = 0.4).opts(toolbar=None)
histogram

#### Variation histogram: density plot
A density plot predicts the count value and smoothens across the x-axis. They're not affected by bins, so they are better at determining the distribution shape

In [ ]:
# Density plot education by gender
df.hvplot.kde('EDUCATION_LOWER_T1', 
                    by='GENDER', alpha=0.5, title='Density plot of education level by gender')

#### Histogram DS1 male - education

In [ ]:
DS_Q_Q_Hist(male.EDUCATION_LOWER_T1)

##### Histogram DS1 female - education

In [ ]:
DS_Q_Q_Hist(female.EDUCATION_LOWER_T1)

The percentage of lower education level seems normally distributed for men, but not for women.

In [ ]:
# .fillna() has the same effect as interpolate
#sel1 = sel1.fillna(method='ffill')
#sel1.tail(5)

In [ ]:
# Check the correlations
#df.corr().abs()

In [ ]:
DS_Q_Q_Plot(male.OSTEOARTHRITIS_T1)

In [ ]:
DS_Q_Q_Hist(male.OSTEOARTHRITIS_T1)

In [ ]:
DS_Q_Q_Plot(female.OSTEOARTHRITIS_T1)

In [ ]:
histogram = male.hvplot.hist('OSTEOARTHRITIS_T1', subplots=True,
                             alpha = 0.4)
histogram

In [ ]:
DS_Q_Q_Hist(female.OSTEOARTHRITIS_T1)

In [ ]:
histogram = female.hvplot.hist('OSTEOARTHRITIS_T1', 
                             alpha = 0.5)
histogram

In [ ]:
shapiro(male.EDUCATION_LOWER_T1)
shapiro(female.EDUCATION_LOWER_T1)
shapiro(female.OSTEOARTHRITIS_T1)
shapiro(female.OSTEOARTHRITIS_T1)

***

***

Some columns have missing values
Use interpolation to fill the missing values.
Method: pad fills in NaNs using existing values.
With this method the limit direction must be forward, 


In [ ]:
# # Sort dataframe and interpolae missing data
# age = age.sort_index()
# age['C_SUM_T1'] = age["C_SUM_T1"].interpolate(method='pad', limit_direction='forward')

For the public health dataset we show the percentage of
participants within their group of aggregation (on zip code or age) whose highest education is
‘lower education’.

Evenly (uniform) data yield a sigmoidal Q-Q plot,

(male education is normally distributed, female education is not)

wilcoxontest/spearman -> not normal distributed

groupby?

***

### Age related

In [ ]:
# Average age of men and women in the dataset is 55
# Use this as a cutoff to split the groups
female_old = female[female['AGE'] > 55]
female_young = female[female['AGE'] <= 55]
male_old = male[male['AGE'] > 55]
male_young = male[male['AGE'] <= 55]

In [ ]:
#correlation matrix
# Heatmap
cor_male = male_young.corr().abs()
#selection.corr().abs()
sns.heatmap(cor_male, annot=True)

In [ ]:
# Density plot education by gender
female_old.hvplot.kde('EDUCATION_LOWER_T1', alpha=0.5)

In [ ]:
DS_Q_Q_Plot(female_old.EDUCATION_LOWER_T1)

In [ ]:
# Density plot education by gender
female_young.hvplot.kde('EDUCATION_LOWER_T1', alpha=0.5)

In [ ]:
#correlation matrix
# Heatmap
cor_male = male_old.corr().abs()
#selection.corr().abs()
sns.heatmap(cor_male, annot=True)

In [ ]:
#correlation matrix
# Heatmap
cor_female = female_young.corr().abs()
#selection.corr().abs()
sns.heatmap(cor_female, annot=True)

In [ ]:
#correlation matrix
# Heatmap
cor_female = female_old.corr().abs()
#selection.corr().abs()
sns.heatmap(cor_female, annot=True)

Average percentage of having a lower education is 32.47 for men

Average percentage of having a lower education is 36.13 for women

In [ ]:
male.GROUP_SIZE_CAT.value_counts()

In [ ]:
female.GROUP_SIZE_CAT.value_counts()